<a href="https://colab.research.google.com/github/georgsmeinung/argentina_demographics/blob/main/Argentina_Demographics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'argentina-demographics-1869-2022:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F5728734%2F9429761%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240918%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240918T194046Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D505c7f01b80a705d7b62ca6c0de87a13b981b36dbc46964c4021d5124400e9d0c5cfcc6d2ffa0b55c376bfcfbbd4515d1318ff978fb93fe453bcac35cc8fca723b0a4ec1513d70adefa798efab0ed7330e9311c7a89acf531f68767a0bf72ed5636d3ada050af5532c707adc6f037be5529c9b4a69aec54e88b60f4af656178d22d9dc71596b1b625877c1bad4b8bb60c001ef6b1133bb115ca0ef98f92124ddff9a307f3db92911b1796c3fefcea5c241a078509906682932d8fdb3e65d9a8c8f83de68ba90fbacb4bbc0386305d2c4fb260ee9f6a6fab2bee257b40ddafa2440bd63349b51d15cc037cd550f6e714201b6a40571283383b3a805a3f4806cee'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


[==================================================] 105763 bytes downloaded
Downloaded and uncompressed: argentina-demographics-1869-2022
Data source import complete.


In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import polars as pl # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        file_origin = os.path.join(dirname, filename)

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Load the CSV into a Polars DataFrame
df = pl.read_csv(file_origin)

# Display the DataFrame
print(df)

/kaggle/input/argentina-demographics-1869-2022/pad_argentina.csv
shape: (19_008, 8)
┌───────┬───────────────┬───────────┬──────┬──────────────┬───────────────────┬───────────┬──────┐
│ censo ┆ iso_3166-2_AR ┆ provincia ┆ sexo ┆ rango_etario ┆ nativo_extranjero ┆ poblacion ┆      │
│ ---   ┆ ---           ┆ ---       ┆ ---  ┆ ---          ┆ ---               ┆ ---       ┆ ---  │
│ i64   ┆ str           ┆ str       ┆ str  ┆ str          ┆ str               ┆ i64       ┆ str  │
╞═══════╪═══════════════╪═══════════╪══════╪══════════════╪═══════════════════╪═══════════╪══════╡
│ 1869  ┆ AR            ┆ ARGENTINA ┆ M    ┆ 05-09        ┆ N                 ┆ 124003    ┆ null │
│ 1869  ┆ AR            ┆ ARGENTINA ┆ M    ┆ 10-14        ┆ N                 ┆ 99081     ┆ null │
│ 1869  ┆ AR            ┆ ARGENTINA ┆ M    ┆ 00-04        ┆ N                 ┆ 129782    ┆ null │
│ 1869  ┆ AR            ┆ ARGENTINA ┆ M    ┆ 15-19        ┆ N                 ┆ 87430     ┆ null │
│ 1869  ┆ AR            ┆